## Recap курса "Введение в машинное обучение"

## Семинар #10

### Екатерина Кондратьева

ekaterina.kondrateva@skoltech.ru

  Программа курса:
  
    1. Обучение с учителем: Линейная и логистическая регрессия. Ядра.
    
    Фолдер:
    
    2. Обучение с учителем: Регуляризация в линейных моделях. Метод Ближайших Соседей (KNN)
    
    Фолдер:
    
    3. Обучение с учителем: Метод опорных векторов (SVM) для задач классификации и регрессии. Kernel SVM
    
    Фолдер:
    
    4. Обучение с учителем: Деревья решений (Decision Trees). Случайный лес (Random Forest).
    
    Фолдер:
    
    5. Оценка качества алгоритмов машинного обучения. Кросс-валидация. Поиск аномалий и артефактов в выборке. 
    
    Фолдер:
    
    6. Обучение без учителя: кластеризация. Снижение размерности данных PCA.
    
    Фолдер: 
    
    7. Отбор и генерация признаков (Feature Engineering). Поиск и оптимизация модели (Grid Search).
    
    Лекция: 
    
    8. Стекинг, вотинг. Градиентный бустинг. Пакеты XGBoost/Catboost/LightGBM  
    Лекция:
    Семинар:

### Построим пайплан для датасета HousePricing:

Данные - https://www.kaggle.com/lider123/housepricing

In [39]:
import pandas as pd
import numpy as np
from scipy.stats import skew
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error
import seaborn as sns

In [36]:
train_data = pd.read_csv("housepricing/train.csv")
test_data = pd.read_csv("housepricing/test.csv")

In [37]:
full_data = pd.concat([train_data,test_data])

C:\Users\admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [38]:
full_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2919 entries, 0 to 1458
Data columns (total 81 columns):
1stFlrSF         2919 non-null int64
2ndFlrSF         2919 non-null int64
3SsnPorch        2919 non-null int64
Alley            198 non-null object
BedroomAbvGr     2919 non-null int64
BldgType         2919 non-null object
BsmtCond         2837 non-null object
BsmtExposure     2837 non-null object
BsmtFinSF1       2918 non-null float64
BsmtFinSF2       2918 non-null float64
BsmtFinType1     2840 non-null object
BsmtFinType2     2839 non-null object
BsmtFullBath     2917 non-null float64
BsmtHalfBath     2917 non-null float64
BsmtQual         2838 non-null object
BsmtUnfSF        2918 non-null float64
CentralAir       2919 non-null object
Condition1       2919 non-null object
Condition2       2919 non-null object
Electrical       2918 non-null object
EnclosedPorch    2919 non-null int64
ExterCond        2919 non-null object
ExterQual        2919 non-null object
Exterior1st      291

In [30]:
categorial_features = [prizn for prizn in full_data.columns if full_data[prizn].dtype.name == 'object']
numeric_features = [prizn for prizn in full_data.columns if full_data[prizn].dtype.name != 'object']
print(categorial_features, numeric_features, sep = '\n')

['Alley', 'BldgType', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'BsmtQual', 'CentralAir', 'Condition1', 'Condition2', 'Electrical', 'ExterCond', 'ExterQual', 'Exterior1st', 'Exterior2nd', 'Fence', 'FireplaceQu', 'Foundation', 'Functional', 'GarageCond', 'GarageFinish', 'GarageQual', 'GarageType', 'Heating', 'HeatingQC', 'HouseStyle', 'KitchenQual', 'LandContour', 'LandSlope', 'LotConfig', 'LotShape', 'MSZoning', 'MasVnrType', 'MiscFeature', 'Neighborhood', 'PavedDrive', 'PoolQC', 'RoofMatl', 'RoofStyle', 'SaleCondition', 'SaleType', 'Street', 'Utilities']
['1stFlrSF', '2ndFlrSF', '3SsnPorch', 'BedroomAbvGr', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtFullBath', 'BsmtHalfBath', 'BsmtUnfSF', 'EnclosedPorch', 'Fireplaces', 'FullBath', 'GarageArea', 'GarageCars', 'GarageYrBlt', 'GrLivArea', 'HalfBath', 'Id', 'KitchenAbvGr', 'LotArea', 'LotFrontage', 'LowQualFinSF', 'MSSubClass', 'MasVnrArea', 'MiscVal', 'MoSold', 'OpenPorchSF', 'OverallCond', 'OverallQual', 'PoolArea', 'SalePrice',

In [31]:
# заполним пропуски числовых данных применяя свои знания предметной области
full_data["SaleType"].fillna(full_data["SaleType"].mode()[0], inplace=True)
full_data["Electrical"].fillna(full_data["Electrical"].mode()[0], inplace=True)
full_data["MSZoning"].fillna(full_data["MSZoning"].mode()[0], inplace=True)
full_data["GarageArea"].fillna(full_data["GarageArea"].median(), inplace=True)
full_data["CentralAir"] = full_data["CentralAir"].map({"Y": 1, "N": 0})
full_data["LotFrontage"].fillna(full_data["LotFrontage"].median(), inplace=True)
full_data["Alley"].fillna("None", inplace=True)
full_data["MasVnrType"].fillna("None", inplace=True)
full_data["MasVnrArea"].fillna(0, inplace=True)

In [32]:
#заполним пропуски числовых данных подряд медианой
for i in numeric_features:
    full_data[i].fillna(full_data[i].median(), inplace=True)

In [33]:
# кодируем переменные типа объекта
full_data = pd.get_dummies(full_data, columns=categorial_features)

X_train, X_test = full_data.iloc[:len(train_data), :], full_data.iloc[len(train_data):, :]

X_train.head()

,1stFlrSF,2ndFlrSF,3SsnPorch,BedroomAbvGr,BsmtFinSF1,BsmtFinSF2,BsmtFullBath,BsmtHalfBath,BsmtUnfSF,EnclosedPorch,...,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,Street_Grvl,Street_Pave,Utilities_AllPub,Utilities_NoSeWa
0,856,854,0,3,706.0,0.0,1.0,0.0,150.0,0,...,0,0,0,0,0,1,0,1,1,0
1,1262,0,0,3,978.0,0.0,0.0,1.0,284.0,0,...,0,0,0,0,0,1,0,1,1,0
2,920,866,0,3,486.0,0.0,1.0,0.0,434.0,0,...,0,0,0,0,0,1,0,1,1,0
3,961,756,0,3,216.0,0.0,1.0,0.0,540.0,272,...,0,0,0,0,0,1,0,1,1,0
4,1145,1053,0,4,655.0,0.0,1.0,0.0,490.0,0,...,0,0,0,0,0,1,0,1,1,0


In [40]:
X_train.shape

(1460, 291)

In [ ]:
params = {
    "n_estimators": [10, 20, 30, 50, 100, 150, 200],
    "max_features": ["auto", "sqrt", "log2", None],
    "max_depth": list(range(2, 11)) + [None]
}
gs = GridSearchCV(estimator=RandomForestRegressor(), param_grid=params, cv=5, scoring="neg_mean_squared_error")
gs.fit(X_train, y_train)
gs.best_params_

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler, MaxAbsScaler
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn import datasets
from pipelinehelper import PipelineHelper

pipe = Pipeline([
    ('scaler', PipelineHelper([
        ('std', StandardScaler()),
        ('max', MaxAbsScaler()),
    ])),
    ('classifier', PipelineHelper([
        ('svm', LinearSVC()),
        ('rf', RandomForestClassifier()),
    ])),
])

params = {
    'scaler__selected_model': pipe.named_steps['scaler'].generate({
        'std__with_mean': [True, False],
        'std__with_std': [True, False],
        'max__copy': [True],
    }),
    'classifier__selected_model': pipe.named_steps['classifier'].generate({
        'svm__C': [0.1, 1.0],
        'rf__n_estimators': [100, 20],
    })
}
grid = GridSearchCV(pipe, params, scoring='accuracy', verbose=1)
grid.fit(X_iris, y_iris)
print(grid.best_params_)
print(grid.best_score_)

### Практикум по *.Git системам:

!['git'](https://i1.wp.com/lanziani.com/slides/gitflow/images/gitflow_1.png?zoom=2)

1. Установка: https://git-scm.com/downloads
2. GUI: https://desktop.github.com/
3. Туториал: https://learngitbranching.js.org/